In [ ]:
from google.colab import files
import pandas as pd
import numpy as np

# Upload file
uploaded = files.upload()

# Read the CSV
df = pd.read_csv("Amazon Sale Report.csv")

# Look at the shape (rows x cols) and first few rows
print(df.shape)
df.head()



In [ ]:
# Standardize column names
df.columns = [c.strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
print(df.columns)


In [ ]:
# Remove space and uniform formatting
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip()


In [ ]:
# Standardize Date format
def parse_date(x):
    try:
        # Let pandas handle multiple formats
        return pd.to_datetime(x, errors="coerce")
    except:
        return pd.NaT

# Apply parser
df["order_date"] = df["date"].apply(parse_date)

# Standardize final format to DD-MM-YYYY
df["order_date"] = df["order_date"].dt.strftime("%d-%m-%Y")


In [ ]:
# Quick sanity check on dates
print("NaT count:", df["order_date"].isna().sum())
print("Min date  :", df["order_date"].min())
print("Max date  :", df["order_date"].max())

In [ ]:
import re

# Standardize city
df["ship_city"] = df["ship_city"].str.strip()                 # remove spaces
df["ship_city"] = df["ship_city"].str.replace(r"[^\w\s]", "", regex=True)  # remove punctuation
df["ship_city"] = df["ship_city"].str.title()                 # title case

# Standardize state
df["ship_state"] = df["ship_state"].str.strip()
df["ship_state"] = df["ship_state"].str.title()


In [ ]:
# Numeric conversion
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")


In [ ]:
# Remove unwanted index column
if "index" in df.columns:
    df = df.drop(columns=["index"])

# Drop empty and non-contributing column
df = df.drop(columns=["new", "pendings", "currency", "ship_postal_code", "ship_country", "fulfilled_by"])

In [ ]:
# --- Normalize the text columns used for logic ---
for col in ["status", "courier_status"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
    else:
        df[col] = ""  # create empty if missing to avoid KeyErrors

# --- Mapping function following your rules exactly ---
def map_status(row):
    s = row["status"]
    c = row["courier_status"]

    # Delivered → Shipped - Delivered to Buyer
    if "shipped - delivered to buyer" in s:
        return "Delivered"

    # Cancelled → Cancelled OR (Shipped AND Courier Status = Cancelled)
    # OR (Pending AND Courier Status = Cancelled)
    if (("cancelled" in s) or ("shipped" in s and "cancelled" in c) or ("pending" in s and "cancelled" in c)):
        return "Cancelled"

    # Returned → Shipped - Returned to Seller OR Shipped - Returning to Seller
    if ("shipped - returned to seller" in s) or ("shipped - returning to seller" in s):
        return "Returned"

    # Problematic → Shipped - Damaged / Lost in Transit / Rejected by Buyer
    if (
        "shipped - damaged" in s
        or "shipped - lost in transit" in s
        or "shipped - rejected by buyer" in s
    ):
        return "Problematic"

    # In Process → Shipped (except Courier Cancelled) OR Shipped - Out for Delivery
    #             OR Shipped - Picked Up OR Pending OR Pending - Waiting for Pick Up OR Shipping
    if (
        ("shipped" in s and "cancelled" not in c)  # any shipped that isn't courier-cancelled
        or ("shipped - out for delivery" in s)
        or ("shipped - picked up" in s)
        or ("pending" in s and "cancelled" not in c) # any pending that isn't courier-cancelled
        or ("pending - waiting for pick up" in s)
        or ("shipping" in s)
    ):
        return "In Process"

    # Fallback
    return "In Process"

# Apply mapping
df["status_cleaned"] = df.apply(map_status, axis=1)


In [ ]:
# Amount/Qty adjustments driven by the cleaned category
# Cancelled / Returned / Problematic → set Amount=0,
zero_out_mask = df["status_cleaned"].isin(["Cancelled", "Returned", "Problematic"])
df.loc[zero_out_mask, ["amount"]] = 0

# Delivered → drop rows with invalid Amount/Qty (NaN or <= 0)
delivered_mask = df["status_cleaned"] == "Delivered"
invalid_delivered = delivered_mask & (
    df["amount"].isna() | (df["amount"] <= 0) | df["qty"].isna() | (df["qty"] <= 0)
)
df = df[~invalid_delivered].copy()

# Final cleanup for numeric columns
df["amount"] = pd.to_numeric(df["amount"], errors="coerce").fillna(0)
df["qty"] = pd.to_numeric(df["qty"], errors="coerce").fillna(0).astype(int)

In [ ]:
# Check value ranges
print(df["qty"].describe())
print(df["amount"].describe())


In [ ]:
# Check duplicates
duplicates = df[df.duplicated(subset=["order_id"], keep=False)]
print(duplicates.head())


In [ ]:
# Drop Duplicates
df = df.drop_duplicates()


In [ ]:
# Quality check
df.isna().sum()

In [ ]:
print(df.shape)


In [ ]:
# Save the cleaned dataframe (df)
# This will act as the main "Fact Table" in Power BI.

OUTPUT_CSV = "fact_sales_cleaned.csv"

# Write to CSV (no index column)
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Cleaned dataset saved as: {OUTPUT_CSV}")
print("Shape (rows x columns):", df.shape)





In [ ]:
# --- Download the cleaned CSV in Colab ---
from google.colab import files

files.download(OUTPUT_CSV)
